In [1]:
import torch
import pandas as pd
import numpy as np
import os
import torch.nn.functional as F
import copy
import matplotlib as mpl
import matplotlib.pyplot as plt
from ipywidgets import interact
from os import listdir
from os.path import isfile, join


path = os.path.join(os.getcwd())
dataset_path = os.path.join(path, "datasets", "full_corrected_dataset", "test")
experiments_path = os.path.join(path, "experiments")

filenames = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]
experiments = [f for f in listdir(experiments_path) if not isfile(join(experiments_path, f))]
models = ["best", "final"]

@interact(test_filename_id=(0,len(filenames)-1,1), experiment=experiments, bestOrFinal=models)
def test_model(test_filename_id=1,experiment=1,bestOrFinal=1):
    filename = filenames[test_filename_id]
    experiment_name = experiment
    model = bestOrFinal
    model_path = os.path.join(experiments_path, experiment_name, "model_weights", "model_" + model + ".pth")

    mapping_path = os.path.join(path, "datasets", "full_corrected_dataset_to_RAW_mapping.csv")
    file_path = os.path.join(dataset_path, filename)

    mapping = pd.read_csv(mapping_path, sep=";")
    data_csv = pd.read_csv(file_path)
    
    original_filename = mapping.loc[mapping['new_id'] == filename].values[:, 2][0]
    tensors = []
    padding_minimum = torch.zeros(180)
    
    if "4cls" in experiment_name:
        names = ["T1", "T2", "T3", "T4"]
    else:
        names = ["T1", "T2", "T3", "T4", "A+E"]

    data = data_csv.iloc[:, 1:].values[:, 0]
    data = data - np.min(data)
    data = data / np.max(data)
    input_tensor = torch.tensor(data, dtype=torch.double)

    tensors.append(input_tensor)
    tensors.append(padding_minimum)
    tensors = torch.nn.utils.rnn.pad_sequence(tensors, batch_first=True)
    tensors = tensors[:-1]

    model = torch.load(model_path,map_location=torch.device('cpu'))
    model.eval()
    outputs = model(tensors)
    softmax_outputs = F.softmax(outputs, dim=1).tolist()[0]
    
    print("Filename: {}\n".format(str(filename)),
         "Original file location: {}\n".format(str(original_filename)),
         "Model: {}\n{}\n".format(str(experiment_name), model),
         "Original class: {}\n".format(filename.split("_")[0]),
         "Predicted class: {}\n".format(names[np.argmax(softmax_outputs, axis=0)]),
         "Predicted probability: {}\n".format(np.max(softmax_outputs)))
    
    plt.figure(figsize=(12,12))
    plt.subplot(211)
    plt.plot(data_csv.iloc[:, 0:].values[:, 0],data_csv.iloc[:, 1:].values[:, 0])

    plt.subplot(212)
    plt.bar(names, softmax_outputs)
    plt.show()


interactive(children=(IntSlider(value=1, description='test_filename_id', max=1850), Dropdown(description='expe…

In [1]:
import pandas as pd
import os

mapping_1 = pd.read_csv(os.path.join(os.getcwd(), "datasets", "RAW_extended_dataset_to_full_extended_dataset_mapping.csv"))
test = pd.read_csv(os.path.join(os.getcwd(), "datasets", "test_corrections.csv"))
train = pd.read_csv(os.path.join(os.getcwd(), "datasets", "train_corrections.csv"))

In [9]:
for row in train.iterrows():
    row = row[1]
    name = row["PULSE"].strip(".csv")
    queried = mapping_1.query("new_id == \'"+name+".csv\'")
    if len(queried) == 0:
        print("Not found "+name)

In [4]:
mapping_1

,Unnamed: 0,new_id,RAW_path
0,0,AE_0.csv,C:\Users\cmata\Desktop\ICPPRoject\icp_17.03\IC...
1,1,AE_1.csv,C:\Users\cmata\Desktop\ICPPRoject\icp_17.03\IC...
2,2,AE_2.csv,C:\Users\cmata\Desktop\ICPPRoject\icp_17.03\IC...
3,3,T1_3.csv,C:\Users\cmata\Desktop\ICPPRoject\icp_17.03\IC...
4,4,T1_4.csv,C:\Users\cmata\Desktop\ICPPRoject\icp_17.03\IC...
...,...,...,...
24268,24268,T4_24268.csv,C:\Users\cmata\Desktop\ICPPRoject\icp_17.03\IC...
24269,24269,T4_24269.csv,C:\Users\cmata\Desktop\ICPPRoject\icp_17.03\IC...
24270,24270,T4_24270.csv,C:\Users\cmata\Desktop\ICPPRoject\icp_17.03\IC...
24271,24271,T4_24271.csv,C:\Users\cmata\Desktop\ICPPRoject\icp_17.03\IC...


In [10]:
mapping_ode = pd.read_csv(os.path.join(os.getcwd(), "datasets", "RAW_extended_dataset_to_full_extended_dataset_mapping.csv"))
mapping_siam = pd.read_csv(os.path.join(os.getcwd(), "datasets", "RAW_siamese_dataset_to_full_siamese_dataset_mapping.csv"))

In [15]:
ode_dict = {}
siam_dict = {}
final_dict = {}
for row in mapping_ode.iterrows():
    row = row[1]
    ID = row["new_id"]
    original_name = row["RAW_path"].split('\\')[-1]
    ode_dict[original_name] = ID
for row in mapping_siam.iterrows():
    row = row[1]
    ID = row["new_id"]
    original_name = row["RAW_path"].split('\\')[-1]
    siam_dict[original_name] = ID
    final_dict[ID] = ode_dict[original_name]
        

In [19]:
out_mapping={
    "siam_name": [],
    "extended_name": []
}
for entry in final_dict:
    out_mapping["siam_name"].append(entry)
    out_mapping["extended_name"].append(final_dict[entry])
df = pd.DataFrame(out_mapping)
df.to_csv(os.path.join(os.getcwd(), "datasets", "full_siamese_dataset_to_full_extended_dataset_mapping.csv"))

In [20]:
df

,siam_name,extended_name
0,AE_0.csv,AE_17307.csv
1,T2_1.csv,T2_17308.csv
2,T2_2.csv,T2_17309.csv
3,T2_3.csv,T2_17310.csv
4,T2_4.csv,T2_17311.csv
...,...,...
21385,T4_21385.csv,T4_24189.csv
21386,T4_21386.csv,T4_24190.csv
21387,T4_21387.csv,T4_24191.csv
21388,T4_21388.csv,T4_24192.csv
